In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import requests

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "python-api-challenge/WeatherPy/output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [7]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

606

In [10]:
#Perform API calls
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

# Build query URL
query_url = url + "appid=" + weather_api_key + "&q=" + city

# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()
print(weather_json)

{'coord': {'lon': -156.47, 'lat': 20.89}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 295.34, 'feels_like': 294.99, 'temp_min': 293.71, 'temp_max': 297.15, 'pressure': 1014, 'humidity': 78}, 'visibility': 16093, 'wind': {'speed': 4.6, 'deg': 50}, 'clouds': {'all': 40}, 'dt': 1594745420, 'sys': {'type': 1, 'id': 7879, 'country': 'US', 'sunrise': 1594741985, 'sunset': 1594789808}, 'timezone': -36000, 'id': 5847411, 'name': 'Kahului', 'cod': 200}


In [ ]:
#Use a for loop to go through cities
#Use try and except for going through cities

#happy_places = response.json()
counter = 0
for place in happy_places["results"]:
    print(place["name"])
    print(place["formatted_address"])
    counter += 1
    if counter == 5
    break